In [5]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

# Filter Deep Dive
This notebook introduces all the available Filters

## Basics

A few basic points

* All Filters are implmented for the RawDataBag and for the JoinedDataBag. Depending for which databag type the filter is implemented its postfix is either `RawFiltere` or `JoinedFilter`.
* All Filters do not copy the dataframes. They just apply filter on existing dataframes, but don't create new ones.
* All Filters have a `filter()` method which takes a databag as parameter and returns a new databag as parameter (again, they dataframes are not copied in the new instance of the databag). However, there is also a filter `filter()` method of the databag itself.
```
a_filter = USDOnlyRawFilter()
a_rawdatabag: RawDataBag = ...

# use the filter() method of the filter..
new_databag = a_filter.filter(a_rawdatabag)

# or use the filter method of the databab
new_databag = a_rawdatabag.filter(a_filter)
```
* Calls to the `filter()` method of the databag can be chained as follows
```
filter1 = USDOnlyRawFilter()
filter2 = OfficialTagsOnlyRawFilter()
a_rawdatabag: RawDataBag = ...

new_databag = a_rawdatabag.filter(filter1).filter(filter2)

```
* The index operator (`[]`) of the databag class is forwarded to the `filter()` method, therefore you can write the previous call as follows:
```
new_databag = a_rawdatabag[filter1][filter2]
```

## Load Demo Databag

In [6]:
from secfsdstools.e_collector.zipcollecting import ZipCollector

databag = ZipCollector.get_zip_by_name('2022q4.zip').collect()

print("sub: ", databag.sub_df.shape)
print("pre: ", databag.pre_df.shape)
print("num: ", databag.num_df.shape)

2023-12-01 16:59:17,087 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2023-12-01 16:59:17,129 [INFO] parallelexecution      items to process: 1
2023-12-01 16:59:17,194 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2022q4.zip
2023-12-01 16:59:18,960 [INFO] parallelexecution      commited chunk: 0


sub:  (23943, 36)
pre:  (1315392, 10)
num:  (2701629, 9)


## `AdshRawFilter`

This filter lets you select the data for certain reports by their adsh number. Just provide the list of the adsh numbers you are interested in in the constructor of the filter.

It operates on all dataframes (sub, pre, and num).

In [3]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

apple_10k_2022_adsh = "0000320193-22-000108"
adsh_filter = AdshRawFilter(adshs=[apple_10k_2022_adsh])

filtered_databag = databag[adsh_filter]

# since we filtered only a single adsh, there is only one line in the sub_df
print("sub: ", filtered_databag.sub_df.shape)
# also the pre_df and the num_df only contain data for a single report
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (1, 36)
pre:  (185, 10)
num:  (503, 9)


## `StmtRawFilter`
This filter filters the data by the stmt column, which defines to which financial statement (BalanceSheet, CashFlow, IncomeStatement, ..) a data point belongs.

The stmt column is part of the pre_df, so only the pre_df is filtered by this filter.

In [4]:
from secfsdstools.e_filter.rawfiltering import StmtRawFilter

stmt_filter = StmtRawFilter(stmts=['BS', 'CF'])

filtered_databag = databag[stmt_filter]

# as expected, only the shape of the pre_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (23943, 36)
pre:  (632959, 10)
num:  (2701629, 9)


## `ReportPeriodRawFilter`
A report always contains datapoints from previous years. For instance, in the BalanceSheet, IncomeStatement, and CashFlow statement also the values for the previous year is displayed.

Moreover, it is common in an annual report that certain datapoints are presented for several previous years.

You can use this filter, if you are only interested in datapoints that belongs to the "period" date of the report (defined in the period column of the sub_df).

The column ddate of the num_df does define the date of the datapoint. So this filter only applies to the num_df.

To demonstrate that, let us first have a look at all the ddates that are present in apple's 2022 10-K report.

In [5]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

apple_10k_2022_adsh = "0000320193-22-000108"
adsh_filter = AdshRawFilter(adshs=[apple_10k_2022_adsh])

apple_10k_2022_databag = databag[adsh_filter]

# first, let's see for what period the report is
# The format is YYYYMMDD, so it is the 30th of September 2022
print(apple_10k_2022_databag.sub_df.period.tolist())

# Next, lets have a look how many different datapoint ddates are in the num_df and lets also count how many datapoints there are
print(apple_10k_2022_databag.num_df.ddate.value_counts())

[20220930]
20220930    226
20210930    171
20200930    102
20190930      3
20220331      1
Name: ddate, dtype: int64


As exepcted, most of the datapoints are from the period of the report (2022). But a significant number of datapoints is from the previous year (2021) and even the year before that (2020).

By applying this filter, only datapoints are filtered that have the same value for ddate as the period column of the report to which they belong.

In [6]:
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter

reportperiod_filter = ReportPeriodRawFilter()

filtered_databag = databag[reportperiod_filter]

# as expected, only the shape of the num_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (23943, 36)
pre:  (1315392, 10)
num:  (1283154, 9)


## `ReportPeriodAndPreviousPeriodRawFilter`
This filter is similar to the `ReportPeriodRawFilter` filter. Instead of just filtering the datapoints of the period of a report, it also filters the datapoints of the previous year.

It also operates only on the num_df.

In [7]:
from secfsdstools.e_filter.rawfiltering import ReportPeriodAndPreviousPeriodRawFilter

reportperiodandprevious_filter = ReportPeriodAndPreviousPeriodRawFilter()

filtered_databag = databag[reportperiodandprevious_filter]

# as expected, only the shape of the num_df is different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (23943, 36)
pre:  (1315392, 10)
num:  (2053920, 9)


## `TagRawFilter` 
This filter filters only datapoints for the defined tags.

it operates on the pre_df and the num_df.

In [8]:
from secfsdstools.e_filter.rawfiltering import TagRawFilter

tag_filter = TagRawFilter(tags=['Assets', 'AssetsCurrent', 'AssetsNoncurrent'])

filtered_databag = databag[tag_filter]

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

print(filtered_databag.pre_df.tag.value_counts())

sub:  (23943, 36)
pre:  (14133, 10)
num:  (29306, 9)
Assets              7519
AssetsCurrent       6081
AssetsNoncurrent     533
Name: tag, dtype: int64


## `MainCoregRawFilter`
Holding companies often also report data of their subsidiaries in their main report. Datapoints of subsidiares contain the name of the subsidiary in the coreg column of the num_df dataframe.

A good example is "AMERICAN ELECTRIC POWER CO INC" (CIK 4904) which has about 7 subsidaries (e.g., Southwestern Electric Power Co, Indiana Michigan Power Co).

If you are not interested in the datapoints of subsidiaries, you can use this filter to remove datapoints of subsidiaries.

This filter operates only on the num_df dataframe.

In [9]:
from secfsdstools.e_filter.rawfiltering import AdshRawFilter

# let us select the 10q from American Electric Power Co Inc
americanelectric_10q_adsh="0000004904-22-000093"
americanelectric_10q_bag = databag[AdshRawFilter(adshs=[americanelectric_10q_adsh])]

# let's see what subsidiaries american electric has
print(americanelectric_10q_bag.num_df.coreg.unique())

print("sub: ", americanelectric_10q_bag.sub_df.shape)
print("pre: ", americanelectric_10q_bag.pre_df.shape)
print("num: ", americanelectric_10q_bag.num_df.shape)

['' 'AppalachianPowerCo' 'IndianaMichiganPowerCo' 'OhioPowerCo'
 'PublicServiceCoOfOklahoma' 'SouthwesternElectricPowerCo' 'AEPTexasInc.'
 'AEPTransmissionCo' 'KentuckyPowerCo']
sub:  (1, 36)
pre:  (425, 10)
num:  (3181, 9)


In [10]:
from secfsdstools.e_filter.rawfiltering import MainCoregRawFilter

# now filter only the datapoints for the holding company
americanelectric_main_10q_bag = americanelectric_10q_bag[MainCoregRawFilter()]

print("sub: ", americanelectric_main_10q_bag.sub_df.shape)
print("pre: ", americanelectric_main_10q_bag.pre_df.shape)
print("num: ", americanelectric_main_10q_bag.num_df.shape)

sub:  (1, 36)
pre:  (425, 10)
num:  (608, 9)


## `OfficialTagsOnlyRawFilter`
Sometimes companies use inofficial tags than the standard us-gaap tags defined by the xbrl taxonomy. In this cases, the version column contains the adsh number of the report, instead something like us-gaap/2022.

Especially if you want to compare reports from different companies, it makes sense to only analyze the official tags.

You can use this filter to do that.

It operates on the pre_df and the num_df dataframe.

In [3]:
# let's check how many different version tags there are in all the reports of a single quarter, and display the first 20
print(len(databag.num_df.version.unique()), databag.num_df.version.unique()[:20])

7231 ['us-gaap/2021' 'srt/2021' 'dei/2021' 'ifrs/2021' 'dei/2021q4' 'dei/2022'
 'us-gaap/2022' 'srt/2022' 'cef/2022' 'ifrs/2022' 'us-gaap-sup/2022q3'
 'srt-sup/2022q3' '0001599916-22-000255' '0001213900-22-060787'
 '0001410578-22-002848' '0001410578-22-002853' '0001410578-22-002850'
 '0001683168-22-006728' '0001213900-22-060856' '0001185185-22-001143']


In [4]:
# it looks as if it is quite common to use inofficial tags as well, so it might definitely make sense to filter only the official tags
from secfsdstools.e_filter.rawfiltering import OfficialTagsOnlyRawFilter

# now filter only the datapoints for the holding company
officialtagsonly_filter = OfficialTagsOnlyRawFilter()

filtered_databag = databag[officialtagsonly_filter]

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (23943, 36)
pre:  (1188914, 10)
num:  (2298378, 9)


## `USDOnlyRawFilter`
International companies often report datapoints also in other currencies than just USD.

And again, if we want to compare the report of different companies, we might be only interested in USD datapoints.

This filter operates on the uom column of the num_df dataframe and removes entries that are not USD.

In [3]:
# let's check how many different "units" there are in all the reports of a single quarter, and display the first 30
print(len(databag.num_df.uom.unique()), databag.num_df.uom.unique()[:30])

1583 ['USD' 'JPY' 'CNY' 'pure' 'shares' 'acre' 'sqft' 'sqm' 'Rate' 'EUR' 'GBP'
 'item' 'numberOfDays' 'security' 'position' 'Position' 'loan' 'Security'
 'investmentPosition' 'Securities' 'bu' 'gal' 'T' 'derivative' 'swap'
 'USD/bbl' 'INR' 'Institutions' 'contract' 'Loan']


In [4]:
from secfsdstools.e_filter.rawfiltering import USDOnlyRawFilter

# now filter only the datapoints for the holding company
usdonly_filter = USDOnlyRawFilter()

filtered_databag = databag[usdonly_filter]

# as expected, only the shape of the pre_df and num_df are different from the unfiltered dataframes
print("sub: ", filtered_databag.sub_df.shape)
print("pre: ", filtered_databag.pre_df.shape)
print("num: ", filtered_databag.num_df.shape)

sub:  (23943, 36)
pre:  (1315392, 10)
num:  (2649205, 9)


In [ ]:
typische Anwendungen -> multiple years -> only current period ... etc

In [ ]:
implement your own filter